In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from pandas_summary import DataFrameSummary
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()

os.chdir('../..')
from src import utils

In [2]:
DATA      = Path('data')
RAW       = DATA/'raw'
INTERIM   = DATA/'interim'
PROCESSED = DATA/'processed'

In [3]:
challenge  = pd.read_csv(RAW/'Challenge_20180423.csv', low_memory=False)
customer   = pd.read_csv(RAW/'Customer.csv', low_memory=False)
isin       = pd.read_csv(RAW/'Isin.csv', low_memory=False)
submission = pd.read_csv(RAW/'sample_submission.csv', low_memory=False)
trade      = pd.read_csv(RAW/'Trade.csv', low_memory=False)

In [4]:
val_friday = pd.read_feather(PROCESSED/'val_180420.feather')

In [15]:
from src.utils import make_val_set
val_thursday = make_val_set(trade[trade.TradeDateKey==20180419], challenge)

In [16]:
val_friday.shape, val_thursday.shape

((487172, 6), (486338, 5))

In [12]:
trades_2018 = trade[trade.TradeDateKey>20180000]

In [14]:
trades_2018.shape

(849110, 8)

In [17]:
train_friday = trades_2018[trades_2018.TradeDateKey<20180420]
train_thursday = trades_2018[trades_2018.TradeDateKey<20180419]

In [21]:
train_friday = train_friday[train_friday.CustomerInterest == 1]
train_thursday = train_thursday[train_thursday.CustomerInterest == 1]

In [24]:
train_friday.head()

,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
1527,20180201,2447,19665,Sell,748160.0,102.650,Done,1.0
1528,20180220,2447,18972,Sell,2959167.0,102.093,NotTraded,1.0
1529,20180108,2554,24873,Buy,2815003.0,103.877,Done,1.0
1530,20180108,2554,19072,Sell,2815003.0,121.963,Done,1.0
1538,20180116,1922,25986,Buy,601586.0,97.984,Done,1.0


In [22]:
train_friday.shape, train_thursday.shape

((346533, 8), (341983, 8))

In [33]:
last_friday = pd.Series(train_friday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['CustomerIdx', 'IsinIdx', 'BuySell'], keep='first')
                    .set_index(['CustomerIdx', 'IsinIdx', 'BuySell'])['TradeDateKey']) \
                 .to_dict()
last_thursday = pd.Series(train_thursday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['CustomerIdx', 'IsinIdx', 'BuySell'], keep='first')
                    .set_index(['CustomerIdx', 'IsinIdx', 'BuySell'])['TradeDateKey']) \
                 .to_dict()

In [52]:
from datetime import date
def date_diff(d1, d2):
    return (date(d1 // 10000, (d1 // 100) % 100, d1 % 100) - \
            date(d2 // 10000, (d2 // 100) % 100, d2 % 100)).days

In [56]:
val_friday['DaysSinceBuySell'] = val_friday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_friday.get((r['CustomerIdx'], r['IsinIdx'], r['BuySell']), 
                                    20170701)), axis=1)
val_thursday['DaysSinceBuySell'] = val_thursday.apply(lambda r: date_diff(r['TradeDateKey'],
                            last_thursday.get((r['CustomerIdx'], r['IsinIdx'], r['BuySell']), 
                                    20170701)), axis=1)

In [57]:
val_friday.head()

,index,TradeDateKey,CustomerIdx,IsinIdx,BuySell,CustomerInterest,DaysSinceBuySell
0,0,20180420,0,21856,Buy,0.0,293
1,1,20180420,0,21856,Sell,0.0,293
2,2,20180420,0,24944,Buy,0.0,293
3,3,20180420,0,24944,Sell,0.0,39
4,4,20180420,0,25992,Buy,0.0,52


In [64]:
val_friday[(val_friday.CustomerInterest==0) & (val_friday.DaysSinceBuySell < 293)]['DaysSinceBuySell'].describe()

count    193071.000000
mean         51.395880
std          31.247081
min           1.000000
25%          24.000000
50%          51.000000
75%          78.000000
max         108.000000
Name: DaysSinceBuySell, dtype: float64

In [65]:
val_friday[(val_friday.CustomerInterest==1) & (val_friday.DaysSinceBuySell < 293)]['DaysSinceBuySell'].describe()

count    1422.000000
mean       27.691983
std        29.573976
min         1.000000
25%         2.000000
50%        15.000000
75%        46.000000
max       108.000000
Name: DaysSinceBuySell, dtype: float64

In [69]:
%%time
last_friday = pd.Series(train_friday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['CustomerIdx', 'IsinIdx'], keep='first')
                    .set_index(['CustomerIdx', 'IsinIdx'])['TradeDateKey']) \
                 .to_dict()
last_thursday = pd.Series(train_thursday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['CustomerIdx', 'IsinIdx'], keep='first')
                    .set_index(['CustomerIdx', 'IsinIdx'])['TradeDateKey']) \
                 .to_dict()
val_friday['DaysSinceTransaction'] = val_friday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_friday.get((r['CustomerIdx'], r['IsinIdx']), 
                                    20170701)), axis=1)
val_thursday['DaysSinceTransaction'] = val_thursday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_thursday.get((r['CustomerIdx'], r['IsinIdx']), 
                                    20170701)), axis=1)

In [70]:
%%time
last_friday = pd.Series(train_friday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['CustomerIdx'], keep='first')
                    .set_index(['CustomerIdx'])['TradeDateKey']) \
                 .to_dict()
last_thursday = pd.Series(train_thursday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['CustomerIdx'], keep='first')
                    .set_index(['CustomerIdx'])['TradeDateKey']) \
                 .to_dict()
val_friday['DaysSinceCustomerActivity'] = val_friday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_friday.get((r['CustomerIdx']), 
                                    20170701)), axis=1)
val_thursday['DaysSinceCustomerActivity'] = val_thursday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_thursday.get((r['CustomerIdx']), 
                                    20170701)), axis=1)

CPU times: user 16.2 s, sys: 92 ms, total: 16.2 s
Wall time: 16.2 s


In [72]:
%%time
last_friday = pd.Series(train_friday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['IsinIdx'], keep='first')
                    .set_index(['IsinIdx'])['TradeDateKey']) \
                 .to_dict()
last_thursday = pd.Series(train_thursday.sort_values('TradeDateKey', ascending=False) \
                    .drop_duplicates(['IsinIdx'], keep='first')
                    .set_index(['IsinIdx'])['TradeDateKey']) \
                 .to_dict()
val_friday['DaysSinceBondActivity'] = val_friday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_friday.get((r['IsinIdx']), 
                                    20170701)), axis=1)
val_thursday['DaysSinceBondActivity'] = val_thursday.apply(lambda r: date_diff(r['TradeDateKey'],
                                    last_thursday.get((r['IsinIdx']), 
                                    20170701)), axis=1)

CPU times: user 15.6 s, sys: 60 ms, total: 15.6 s
Wall time: 15.6 s


In [82]:
# maybe log for smoothness
pd.concat([val_friday[(val_friday.CustomerInterest==1) & (val_friday.DaysSinceBuySell < 293)]['DaysSinceBuySell'].describe(),
           val_friday[(val_friday.CustomerInterest==1) & (val_friday.DaysSinceTransaction < 293)]['DaysSinceTransaction'].describe(),
           val_friday[(val_friday.CustomerInterest==1) & (val_friday.DaysSinceCustomerActivity < 293)]['DaysSinceCustomerActivity'].describe(),
           val_friday[(val_friday.CustomerInterest==1) & (val_friday.DaysSinceBondActivity < 293)]['DaysSinceBondActivity'].describe()], axis=1)

,DaysSinceBuySell,DaysSinceTransaction,DaysSinceCustomerActivity,DaysSinceBondActivity
count,1422.000000,1648.000000,3184.000000,3121.000000
mean,27.691983,26.627427,1.528580,6.173021
std,29.573976,28.573271,4.191587,11.666006
min,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,1.000000,1.000000
50%,15.000000,15.000000,1.000000,2.000000
75%,46.000000,46.000000,1.000000,7.000000
max,108.000000,108.000000,108.000000,106.000000


In [83]:
val_friday.to_feather(PROCESSED/'friday_datediffs.feather')
val_thursday.to_feather(PROCESSED/'thursday_datediffs.feather')

In [85]:
from src.utils import add_datediffs

In [86]:
%%time
df_test = make_val_set(trade[trade.TradeDateKey==20180420], challenge)

In [108]:
%%time
add_datediffs(df_test, trade[trade.TradeDateKey>20180000])

CPU times: user 38.9 s, sys: 172 ms, total: 39 s
Wall time: 39.4 s


In [106]:
val_friday.head()

,index,TradeDateKey,CustomerIdx,IsinIdx,BuySell,CustomerInterest,DaysSinceBuySell,DaysSinceTransaction,DaysSinceCustomerActivity,DaysSinceBondActivity
0,0,20180420,0,21856,Buy,0.0,293,293,1,30
1,1,20180420,0,21856,Sell,0.0,293,293,1,30
2,2,20180420,0,24944,Buy,0.0,293,39,1,30
3,3,20180420,0,24944,Sell,0.0,39,39,1,30
4,4,20180420,0,25992,Buy,0.0,52,52,1,9


In [115]:
test = challenge.copy()
test['TradeDateKey'] = test['DateKey']

In [116]:
%%time
add_datediffs(test, trade[trade.TradeDateKey>20180000])

CPU times: user 42 s, sys: 160 ms, total: 42.2 s
Wall time: 42.2 s


In [117]:
test.head()

,PredictionIdx,DateKey,CustomerIdx,IsinIdx,BuySell,CustomerInterest,TradeDateKey,DaysSinceBuySell,DaysSinceTransaction,DaysSinceCustomerActivity,DaysSinceBondActivity
0,a1e0d80784,20180423,1856,13323,Buy,NaN,20180423,296,296,3,3
1,c2cc6cc2a8,20180423,1856,9230,Buy,NaN,20180423,14,14,3,3
2,a8e94f6344,20180423,1780,9157,Buy,NaN,20180423,296,296,3,296
3,758bae1e35,20180423,2129,9131,Buy,NaN,20180423,296,296,3,11
4,02ab378ee8,20180423,1758,7151,Buy,NaN,20180423,296,296,3,33


In [118]:
test.to_feather(PROCESSED/'test_datediffs.feather')